In [1]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 614188219315928546
 xla_global_id: -1,
 name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 14343274496
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 17151689243788592156
 physical_device_desc: "device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5"
 xla_global_id: 416903419]

In [2]:
!pip install torch
!pip install torchvision
!pip install scikit-learn
!pip install scipy
!pip install ekphrasis
!pip install openai-clip

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.8/83.8 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.8/52.8 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 kB 7.6 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 18.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for openai-clip: filename=open

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import pandas as pd
import numpy as np
import clip
import torch.optim as optim
import time
from sklearn import metrics
import argparse
import numpy as np
from torch import nn
from torch.nn import functional as F
import torch
import sys
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# Insert the directory
sys.path.insert(0,'/content/drive/MyDrive/MamiGit/mami')
import random
from helper_functions import *
from text_normalizer import *

In [5]:
import os
os.getcwd()

'/content'

In [6]:
## Dataset Loading
# tr_df = pd.read_csv('/content/drive/MyDrive/MamiProject/files/small_training.csv', sep=',') # use '/t' for main training data
# tr_df = pd.read_csv('/content/drive/MyDrive/MamiProject/files/training.csv', sep='\t') # use '/t' for main training data
tr_df = pd.read_csv('/content/drive/MyDrive/MamiGit/mami/data/train.tsv', sep='\t') # use '/t' for main training data
# tr_df = tr_df.head(100)
vl_df = pd.read_csv('/content/drive/MyDrive/MamiGit/mami/data/validation.tsv', sep='\t') # use '/t' for main training data
# vl_df = vl_df.head(100)
ts_df = pd.read_csv('/content/drive/MyDrive/MamiGit/mami/data/test.tsv', sep='\t') # use '/t' for main training data
# ts_df = ts_df.head(20)

In [7]:
print(tr_df.head())

   file_name  label  shaming  stereotype  objectification  violence  \
0   8716.jpg      1        0           1                1         0   
1   3066.jpg      1        1           1                0         0   
2   6038.jpg      0        0           0                0         0   
3  10861.jpg      1        0           0                1         0   
4  11198.jpg      0        0           0                0         0   

                                                text  
0  GETS MARRIED TO GIRL OF HIS DREAMS SHE DOESN'T...  
1  When your mama don't change yo diaper for 19 y...  
2  Some people want a big house, a fast car, and ...  
3     FAP FAP FAP FAP FAP FAP memecenter Meme Center  
4              I RAISE. A I CALL. I FOLD. I'M ALL IN  


In [8]:
tr_df.rename(columns = {'Text Transcription':'text'}, inplace = True)
tr_df.rename(columns = {'misogynous':'label'}, inplace = True)

my_list = tr_df.columns.values.tolist()
print (my_list)

['file_name', 'label', 'shaming', 'stereotype', 'objectification', 'violence', 'text']


In [9]:
print(vl_df.head())
print(len(vl_df))


  file_name  label  shaming  stereotype  objectification  violence  \
0  1377.jpg      1        0           1                0         0   
1  5498.jpg      1        0           1                0         0   
2  5697.jpg      1        0           1                0         0   
3  5182.jpg      1        0           0                0         0   
4  6162.jpg      1        0           1                0         0   

                                                text  
0  IF WOMEN WANT EQUAL PAY THEY SHOULD DO EQUAL W...  
1                  IF A WOMAN WERE TO DRIVE YOUR CAR  
2                           When a woman is a driver  
3  SEXY BITCHES? Ba GIVE THEM BACON TO MAKE THEM ...  
4  When you reach peak feminism FEMINIST Just Bec...  
1000


In [10]:
print(ts_df.head)

<bound method NDFrame.head of      file_name  label  shaming  stereotype  objectification  violence  \
0    15236.jpg      0        0           0                0         0   
1    15805.jpg      1        0           1                1         0   
2    16254.jpg      0        0           0                0         0   
3    16191.jpg      1        0           1                1         0   
4    15952.jpg      0        0           0                0         0   
..         ...    ...      ...         ...              ...       ...   
995  15591.jpg      1        0           1                1         0   
996  15049.jpg      0        0           0                0         0   
997  15363.jpg      1        0           1                1         0   
998  15199.jpg      0        0           0                0         0   
999  15853.jpg      0        0           0                0         0   

                                                  text  
0    FACEBOOK SINGLES GROUPS BELIKE 

In [11]:
seed = 42
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
# Arguments
batch_size=64
init_lr=1e-4
epochs=10
max_length=77
lr=1e-5
vmodel='vit14'

In [12]:
## Pre-trained Stream Models
clip_nms = {'vit14':'ViT-L/14'}
clip_dim = {'vit14': 768}
clip_model, _ = clip.load(clip_nms[vmodel],jit=False)
input_resolution = clip_model.visual.input_resolution
clip_model.float().eval()
clip_model = nn.DataParallel(clip_model)
dim = clip_dim[vmodel]

100%|███████████████████████████████████████| 890M/890M [00:10<00:00, 91.9MiB/s]


In [13]:
## Transforms
transform_config = {'train': transforms.Compose([
        transforms.RandomResizedCrop(input_resolution, interpolation=transforms.InterpolationMode.BICUBIC),
        transforms.RandomHorizontalFlip(),
        # transforms.ColorJitter(brightness=0.2, contrast=0.1, saturation=0.1, hue=0.2),
        transforms.RandomGrayscale(p=0.1),
        # transforms.RandomPerspective(),
        transforms.RandomRotation(degrees=10),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.48145466, 0.4578275, 0.40821073],
                                    std=[0.26862954, 0.26130258, 0.27577711])
    ]), 
    'test': transforms.Compose([
        transforms.Resize((input_resolution,input_resolution), interpolation=transforms.InterpolationMode.BICUBIC),
        # transforms.CenterCrop(clip_model.visual.input_resolution),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.48145466, 0.4578275, 0.40821073],
                                    std=[0.26862954, 0.26130258, 0.27577711])
    ])
}

In [14]:
# !pip install simpletokenizer
# import simpletokenizer as _Tokenizer

from clip.simple_tokenizer import SimpleTokenizer
_tokenizer = SimpleTokenizer()

def tokenize(text, context_length: int = 77):

    sot_token = _tokenizer.encoder["<|startoftext|>"]
    eot_token = _tokenizer.encoder["<|endoftext|>"]
    # all_tokens = _tokenizer.encode(text)
    tokens = [sot_token] + _tokenizer.encode(text)[:context_length-2] + [eot_token]
    result = torch.zeros(context_length, dtype=torch.long)
    mask = torch.zeros(context_length, dtype=torch.long)
    result[:len(tokens)] = torch.tensor(tokens)
    mask[:len(tokens)] = 1

    return result, mask

In [15]:
## Dataset
tr_data = CustomDatasetFixed(tr_df, 'training', transform_config['test'], preprocess, tokenize, max_length)
vl_data = CustomDatasetFixed(vl_df, 'training', transform_config['test'], preprocess, tokenize, max_length)
ts_data = CustomDatasetFixed(ts_df, 'test', transform_config['test'], preprocess, tokenize, max_length)

In [16]:
print(tr_data)

In [17]:
class MMNetwork(nn.Module):
    def __init__(self, vdim, tdim, n_cls):
        super(MMNetwork, self).__init__()
        # print("vdim", vdim)
        # print("tdim", tdim)
        ## Linear layer for ResNet features
        self.vfc = nn.Linear(vdim, 256)

        ## Single Layer Bi-directional RNN with GRU cells. Projects 768 to 512 
        self.bigru = nn.LSTM(tdim, 256, 1, bidirectional=False, batch_first=True, bias=False)

        ## Concatenated Image and Text goes through this multi-layer network
        self.mfc1 = nn.Linear(512, 256)
        # self.mfc2 = nn.Linear(512, 256)
        # self.mfc3 = nn.Linear(256, 128)
        # self.mfc4 = nn.Linear(256, 128)

        self.cf1 = nn.Linear(256, 1)
        self.cf2 = nn.Linear(256, 1)
        self.cf3 = nn.Linear(256, 1)
        self.cf4 = nn.Linear(256, 1)
        self.cf5 = nn.Linear(256, 1)


        self.act = nn.ReLU()    ## ReLU
        self.vdp = nn.Dropout(0.2)
        self.tdp = nn.Dropout(0.2)

    def forward(self, vx, tx, masks=None):
        vx = self.vdp(self.act(self.vfc(vx)))
        hidden_tx,_ = self.bigru(tx)
        # Concatenate Visual and Textual output
        mx = torch.cat((vx, self.tdp(hidden_tx)), dim=1) # changed this

        mx = self.act(self.mfc1(mx))
        # mx = self.act(self.mfc2(mx))
        # mx = self.relu(self.mfc3(mx))
        # mx = self.relu(self.mfc4(mx))

        return torch.sigmoid(self.cf1(mx)), torch.sigmoid(self.cf2(mx)), torch.sigmoid(self.cf3(mx)), \
                torch.sigmoid(self.cf4(mx)), torch.sigmoid(self.cf5(mx))

In [18]:
def evaluate(model, loader, test_flag):
    model.eval()
    test_loss = 0
    all_preds1 = []
    all_labels1 = []
    all_preds2 = []
    all_labels2 = []
    all_preds3 = []
    all_labels3 = []
    all_preds4 = []
    all_labels4 = []
    all_preds5 = []
    all_labels5 = []

    with torch.no_grad():
        for img_inps, txt_tokens, masks, labels1, labels2, labels3, labels4, labels5 in loader:

            img_inps, txt_tokens, masks, labels1, labels2, labels3, labels4, labels5 = img_inps.to(device), \
                txt_tokens.to(device), masks.to(device), labels1.to(device), labels2.to(device), labels3.to(device), \
                labels4.to(device), labels5.to(device)

            img_feats = clip_model.module.encode_image(img_inps)
            txt_feats = clip_model.module.encode_text(txt_tokens) # changed this
            # _, txt_feats = clip_model.module.encode_text(txt_tokens)

            outputs1, outputs2, outputs3, outputs4, outputs5 = model(img_feats, txt_feats, masks)

            preds1 = (outputs1>0.5).int()
            preds2 = (outputs2>0.5).int()
            preds3 = (outputs3>0.5).int()
            preds4 = (outputs4>0.5).int()
            preds5 = (outputs5>0.5).int()
            
            test_loss += (criterion(outputs1, labels1.unsqueeze(1).float()).item() + criterion(outputs2, labels2.unsqueeze(1).float()).item() + \
                            criterion(outputs3, labels3.unsqueeze(1).float()).item() + criterion(outputs4, labels4.unsqueeze(1).float()).item() + \
                            criterion(outputs5, labels5.unsqueeze(1).float()).item())

            all_preds1.extend(preds1.cpu().numpy().flatten())
            all_labels1.extend(labels1.cpu().numpy().flatten())
            all_preds2.extend(preds2.cpu().numpy().flatten())
            all_labels2.extend(labels2.cpu().numpy().flatten())
            all_preds3.extend(preds3.cpu().numpy().flatten())
            all_labels3.extend(labels3.cpu().numpy().flatten())
            all_preds4.extend(preds4.cpu().numpy().flatten())
            all_labels4.extend(labels4.cpu().numpy().flatten())
            all_preds5.extend(preds5.cpu().numpy().flatten())
            all_labels5.extend(labels5.cpu().numpy().flatten())

        acc = metrics.accuracy_score(all_labels1, all_preds1)
        f1_1 = metrics.f1_score(all_labels1, all_preds1, average='macro')
        f1_2 = metrics.f1_score(all_labels2, all_preds2, average='macro')
        f1_3 = metrics.f1_score(all_labels3, all_preds3, average='macro')
        f1_4 = metrics.f1_score(all_labels4, all_preds4, average='macro')
        f1_5 = metrics.f1_score(all_labels5, all_preds5, average='macro')

        #Get all the incorrect predictions
        if(test_flag):
          # Open a file to write the indexes of non-matching labels
          with open('/content/drive/MyDrive/non_matching_indexes.txt', 'w') as f:
            # Loop through the indexes of the labels in the first list
            for i in range(len(all_labels1)): 
              # Check if the label in the first list does not match the label in the second list
              if all_labels1[i] != all_preds1[i]:
                # Write the index to the file
                f.write(str(i) + '\n')
    return test_loss/len(loader), acc, f1_1, f1_2, f1_3, f1_4, f1_5

In [19]:
def train(model, optimizer, lr_scheduler, num_epochs):
    test_flag=False
    since = time.time()

    best_model = model
    best_acc = 0.0
    best_val_loss = 100
    best_epoch = 0
    best_f1 = 0

    for epoch in range(1, num_epochs+1):
        print('Epoch {}/{}'.format(epoch, num_epochs))
        print('-' * 10)

        since2 = time.time()

        model.train()  # Set model to training mode

        running_loss = 0.0
        running_corrects = 0

        tot = 0.0
        cnt = 0
        # Iterate over data.
        for img_inps, txt_tokens, masks, labels1, labels2, labels3, labels4, labels5 in tr_loader:

            img_inps, txt_tokens, masks, labels1, labels2, labels3, labels4, labels5 = img_inps.to(device), \
                txt_tokens.to(device), masks.to(device), labels1.to(device), labels2.to(device), labels3.to(device), \
                labels4.to(device), labels5.to(device)

            # zero the parameter gradients
            optimizer.zero_grad()
            # forward
            with torch.no_grad():
                img_feats = clip_model.module.encode_image(img_inps)
                txt_feats = clip_model.module.encode_text(txt_tokens) # changed this

            outputs1, outputs2, outputs3, outputs4, outputs5 = model(img_feats, txt_feats, masks)
            preds1 = (outputs1>0.5).int()

            loss = criterion(outputs1, labels1.unsqueeze(1).float()) + criterion(outputs2, labels2.unsqueeze(1).float()) + \
                    criterion(outputs3, labels3.unsqueeze(1).float()) + criterion(outputs4, labels4.unsqueeze(1).float()) + \
                    criterion(outputs5, labels5.unsqueeze(1).float())

            # backward + optimize
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            optimizer.step()

            # statistics
            running_loss += loss.item()
            running_corrects += torch.sum(preds1 == labels1.data.view_as(preds1)).item()
            tot += len(labels1)

            if cnt % 40 == 0:
                print('[%d, %5d] loss: %.4f, Acc: %.2f' %
                      (epoch, cnt + 1, loss.item(), (100.0 * running_corrects) / tot))

            cnt = cnt + 1

        if scheduler:
            lr_scheduler.step()


        train_loss = running_loss / len(tr_loader)
        train_acc = running_corrects * 1.0 / (len(tr_loader.dataset))

        print('Training Loss: {:.6f} Acc: {:.2f}'.format(train_loss, 100.0 * train_acc))

        test_loss, test_acc, test_f1_1, test_f1_2, test_f1_3, test_f1_4, test_f1_5 = evaluate(model, vl_loader, test_flag)

        print('Epoch: {:d}, Val Loss: {:.4f}, Acc: {:.2f}, F1_1: {:.2f}, F1_2: {:.2f}, F1_3: {:.2f}, F1_4: {:.2f}, F1_5: {:.2f}'.format(epoch, test_loss,test_acc*100, test_f1_1*100, test_f1_2*100, \
                 test_f1_3*100, test_f1_4*100, test_f1_5*100))


        # deep copy the model
        if test_f1_1 >= best_f1:
            best_acc = test_acc
            best_val_loss = test_loss
            best_model = copy.deepcopy(model)
            best_epoch = epoch
            best_f1 = test_f1_1

    time_elapsed2 = time.time() - since2
    print('Epoch complete in {:.0f}m {:.0f}s'.format(
        time_elapsed2 // 60, time_elapsed2 % 60))

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))

    return best_model, best_epoch

In [20]:
from torch.utils.data import DataLoader, Dataset, sampler
tr_loader = DataLoader(tr_data, num_workers=4, batch_size=batch_size)
vl_loader = DataLoader(vl_data, num_workers=2, batch_size=batch_size)
ts_loader = DataLoader(ts_data, num_workers=2, batch_size=batch_size)

## Model
model = MMNetwork(dim, dim, 1)
model.to(device)
print(model)

optimizer = optim.Adam(model.parameters(), init_lr, betas=(0.99,0.98), weight_decay=1e-4)
criterion = nn.BCELoss()
num_train_steps = int(len(tr_data)/batch_size)*epochs
num_warmup_steps = int(0.1*num_train_steps)
print(num_train_steps, num_warmup_steps)    ## Print Number of total and warmup steps
# scheduler = transformers.get_linear_schedule_with_warmup(optimizer, num_warmup_steps=num_warmup_steps, num_training_steps=num_train_steps)
scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, [5,10,15],gamma=0.5)
# scheduler = None
import time
start_time = time.time()
model_ft, best_epoch = train(model, optimizer, scheduler, num_epochs=epochs)
print("--- %s seconds ---" % (time.time() - start_time))

# torch.save(model_ft.state_dict(), 'saved_models/trained_model_%s.pt'%(args.net, vmodel))
test_flag=False
vl_loss, vl_acc, vl_f1, _, _, _, _ = evaluate(model_ft, vl_loader, test_flag)
print('Validation best epoch: %d, Val Loss: %.4f, ACC: %.2f, F1: %.2f'%(best_epoch, np.round(vl_loss,4), vl_acc*100, vl_f1*100))
test_flag=True
ts_loss, ts_acc, ts_f1, ts_f2, ts_f3, ts_f4, ts_f5 = evaluate(model_ft, ts_loader, test_flag)
print('Test results:, Test Loss: %.4f, ACC: %.2f, F1: %.2f, F2: %.2f, F3: %.2f, F4: %.2f, F5: %.2f'%(np.round(ts_loss,4), ts_acc*100, ts_f1*100, ts_f2*100, ts_f3*100, ts_f4*100, ts_f5*100))

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:561: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


MMNetwork(
  (vfc): Linear(in_features=768, out_features=256, bias=True)
  (bigru): LSTM(768, 256, bias=False, batch_first=True)
  (mfc1): Linear(in_features=512, out_features=256, bias=True)
  (cf1): Linear(in_features=256, out_features=1, bias=True)
  (cf2): Linear(in_features=256, out_features=1, bias=True)
  (cf3): Linear(in_features=256, out_features=1, bias=True)
  (cf4): Linear(in_features=256, out_features=1, bias=True)
  (cf5): Linear(in_features=256, out_features=1, bias=True)
  (act): ReLU()
  (vdp): Dropout(p=0.2, inplace=False)
  (tdp): Dropout(p=0.2, inplace=False)
)
1400 140
Epoch 1/10
----------
[1,     1] loss: 3.5315, Acc: 59.38
[1,    41] loss: 2.2288, Acc: 57.93
[1,    81] loss: 2.3039, Acc: 64.18
[1,   121] loss: 2.2619, Acc: 68.07
Training Loss: 2.511813 Acc: 69.60
Epoch: 1, Val Loss: 2.1033, Acc: 82.20, F1_1: 82.20, F1_2: 87.70, F1_3: 75.20, F1_4: 78.10, F1_5: 90.00
Epoch 2/10
----------


/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:561: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


[2,     1] loss: 2.0427, Acc: 79.69
[2,    41] loss: 1.7113, Acc: 82.16
[2,    81] loss: 1.4480, Acc: 82.47
[2,   121] loss: 1.8147, Acc: 83.17
Training Loss: 1.853256 Acc: 83.17
Epoch: 2, Val Loss: 1.7068, Acc: 85.20, F1_1: 85.20, F1_2: 87.70, F1_3: 76.70, F1_4: 84.70, F1_5: 90.00
Epoch 3/10
----------


/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:561: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


...

[10,     1] loss: 1.2585, Acc: 89.06
[10,    41] loss: 1.1373, Acc: 88.87
[10,    81] loss: 0.8807, Acc: 89.16
[10,   121] loss: 1.0975, Acc: 89.02
Training Loss: 1.284678 Acc: 89.07
Epoch: 10, Val Loss: 1.3967, Acc: 89.20, F1_1: 89.20, F1_2: 90.40, F1_3: 81.30, F1_4: 87.00, F1_5: 92.30
Epoch complete in 8m 31s
Training complete in 88m 21s
--- 5301.000227689743 seconds ---


/usr/local/lib/python3.10/dist-packages/torch/nn/modules/rnn.py:812: UserWarning: RNN module weights are not part of single contiguous chunk of memory. This means they need to be compacted at every call, possibly greatly increasing memory usage. To compact weights again call flatten_parameters(). (Triggered internally at ../aten/src/ATen/native/cudnn/RNN.cpp:982.)
  result = _VF.lstm(input, hx, self._flat_weights, self.bias, self.num_layers,


Validation best epoch: 6, Val Loss: 1.4221, ACC: 89.60, F1: 89.60


/usr/local/lib/python3.10/dist-packages/torch/nn/modules/rnn.py:812: UserWarning: RNN module weights are not part of single contiguous chunk of memory. This means they need to be compacted at every call, possibly greatly increasing memory usage. To compact weights again call flatten_parameters(). (Triggered internally at ../aten/src/ATen/native/cudnn/RNN.cpp:982.)
  result = _VF.lstm(input, hx, self._flat_weights, self.bias, self.num_layers,


Test results:, Test Loss: 2.1456, ACC: 74.40, F1: 74.40, F2: 86.30, F3: 72.60, F4: 81.70, F5: 88.30
